**Fully cleaning the dataset**

**Importing data**


In [1]:
# imports
from IPython.display import clear_output
from PIL import Image
import os
import json
# clear
clear_output(wait=True)
# installing kaggle
!pip install -q kaggle
# user and key info
info = {"username":"jackbellamy","key":"3442898ba5e904ac3a48eb5b59398805"}
# opening file
json_object = json.dumps(info, indent=4)
with open("kaggle.json", "w") as outfile:
    outfile.write(json_object)
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# copying data to content file of kaggle which can be found on the left hand side of the screen
if not (os.path.exists("/content/nfl-big-data-bowl-2022")):
  !kaggle competitions download -c nfl-big-data-bowl-2022
  !mkdir nfl-big-data-bowl-2022
  !unzip nfl-big-data-bowl-2022.zip -d nfl-big-data-bowl-2022

 98% 769M/783M [00:22<00:00, 35.1MB/s]
100% 783M/783M [00:22<00:00, 35.8MB/s]
Archive:  nfl-big-data-bowl-2022.zip
  inflating: nfl-big-data-bowl-2022/PFFScoutingData.csv  
  inflating: nfl-big-data-bowl-2022/games.csv  
  inflating: nfl-big-data-bowl-2022/players.csv  
  inflating: nfl-big-data-bowl-2022/plays.csv  
  inflating: nfl-big-data-bowl-2022/tracking2018.csv  
  inflating: nfl-big-data-bowl-2022/tracking2019.csv  
  inflating: nfl-big-data-bowl-2022/tracking2020.csv  


**Serialisation**

In [2]:
import pandas as pd
# reading the data to variables
scouting = pd.read_csv("/content/nfl-big-data-bowl-2022/PFFScoutingData.csv")
games = pd.read_csv("/content/nfl-big-data-bowl-2022/games.csv")
players = pd.read_csv("/content/nfl-big-data-bowl-2022/players.csv")
plays = pd.read_csv("/content/nfl-big-data-bowl-2022/plays.csv")
tracking2018 = pd.read_csv("/content/nfl-big-data-bowl-2022/tracking2018.csv")
tracking2019 = pd.read_csv("/content/nfl-big-data-bowl-2022/tracking2019.csv")
tracking2020 = pd.read_csv("/content/nfl-big-data-bowl-2022/tracking2020.csv")

**Generalised cleaning functions**

**Abbreviations to full names**

In [3]:
def abbrvs(csv,column,abbrv,fullname):
  # new names
  type_name = []
  # looping through
  for i in csv[column]:
    # filtering out NaN values
    if i in abbrv:
      index = abbrv.index(i)
      type_name.append(fullname[index])
    else:
      type_name.append(i)
  return type_name

**Team and number splitting**

In [4]:
def team_number_splitting(csv,column,symbol):
  # splitting to team and number
  split_team = []
  split_numbers = []
  # splitting the data
  split = csv[column].str.split(symbol, expand = True)
  # loop for each
  for i in range(0,len(csv)):
    # specific data
    split.iloc[i]
    # number of NaN
    nan_num = split.iloc[i].isna().sum()
    max = len(split.iloc[i])
    if max-nan_num == 0:
      split_team.append(csv[column][i])
      split_numbers.append(csv[column][i])
    else:
      info = []
      for j in range(0,(max-nan_num)):
        split_i = split[j][i].split(" ")
        info.append(split_i)
      # team name
      split_team.append(info[0][0])
      # numbers
      nums = []
      for k in range(0,max-nan_num):
        num = info[k][1]
        nums.append(num)
      split_numbers.append(nums)
  return split_team,split_numbers

**Number splitting**

In [5]:
def number_splitting(csv,column,symbol,n):
  split = csv[column].str.split(symbol, n = n, expand = True)
  return split

**Pandas to number**

In [6]:

def pd_to_num(csv,split,column_names):
  split.columns = column_names
  # adding columns
  for name in column_names:
    csv[name] = pd.to_numeric(split[name])
  return

**Scouting** 


In [7]:
# before
scouting.head()

,gameId,playId,snapDetail,snapTime,operationTime,hangTime,kickType,kickDirectionIntended,kickDirectionActual,returnDirectionIntended,returnDirectionActual,missedTackler,assistTackler,tackler,kickoffReturnFormation,gunners,puntRushers,specialTeamsSafeties,vises,kickContactType
0,2018090600,37,NaN,NaN,NaN,3.85,D,R,R,NaN,NaN,NaN,NaN,NaN,8-0-2,NaN,NaN,PHI 23; PHI 27,NaN,NaN
1,2018090600,366,OK,0.84,2.12,4.46,N,C,C,C,R,PHI 57,NaN,PHI 54,NaN,PHI 18; PHI 29,NaN,NaN,ATL 83; ATL 27; ATL 34; ATL 21,CC
2,2018090600,658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PHI 58,NaN,NaN
3,2018090600,677,NaN,NaN,NaN,4.06,D,R,R,C,C,ATL 83,ATL 22,ATL 27,8-0-2,NaN,NaN,ATL 17; ATL 22,NaN,NaN
4,2018090600,872,OK,0.84,2.00,4.35,N,C,L,NaN,NaN,NaN,NaN,NaN,NaN,PHI 18; PHI 29,ATL 85,ATL 37,ATL 83; ATL 34; ATL 21,BF


In [ ]:
# abbreviations: 'kickType', 'kickDirectionIntended', 'kickDirectionActual', 'returnDirectIntended', 'returnDirectionActual', 'KickTypeContact'
# team number splitting: 'missedTackler', 'specialTeamsSafeties', 'Vises'
# number splitting: 'kickoffReturnFormation'

In [8]:
# abbreviations
# 'kickType'
csv = scouting
column = "kickType"
kick_types = ['D','F','K','O','P','Q','S','B','N','R','A']
kick_type_names = ['Deep','Flat','Free','Obvious','Pooch','Squib','Surprise','Deep','Normal','Rugby','Aussie']
# calling function
kick_type_fullnames = abbrvs(csv,column,kick_types,kick_type_names)
# adding column
scouting["kick_type_name"] = kick_type_fullnames

# 'kickDirectionIntended', 'kickDirectionActual', 'returnDirectIntended', 'returnDirectionActual', 'KickTypeContact'

In [9]:
# team and numbers
# 'missedTackler'
csv = scouting

column = "missedTackler"
symbol = "; "
# calling function
missed_tackler_team, missed_tackler_numbers = team_number_splitting(csv,column,symbol)
# adding
scouting["missed_tackler_team"] = missed_tackler_team
scouting["missed_tackler_numbers"] = missed_tackler_numbers

# 'specialTeamsSafeties'
column = "specialTeamsSafeties"
symbol = "; "
# calling function
specialTeamsSafeties_team, specialTeamsSafeties_numbers = team_number_splitting(csv,column,symbol)
# adding
scouting["special_teams_safeties_team"] = specialTeamsSafeties_team
scouting["special_teams_safeties_numbers"] = specialTeamsSafeties_numbers

# 'Vises'
column = "vises"
symbol = "; "
# calling function
vises_team, vises_numbers = team_number_splitting(csv,column,symbol)
# adding
scouting["vises_team"] = vises_team
scouting["vises_numbers"] = vises_numbers

In [10]:
# number splitting
# 'kickoffReturnFormation'
csv = scouting
column = "kickoffReturnFormation"
symbol = "-"
n = 2
column_names = ['formation_front','formation_mid','formation_back']
# calling function
formation_split = number_splitting(csv,column,symbol,n)
# calling function
pd_to_num(csv,formation_split,column_names)

In [11]:
# after
scouting.head()

,gameId,playId,snapDetail,snapTime,operationTime,hangTime,kickType,kickDirectionIntended,kickDirectionActual,returnDirectionIntended,...,kick_type_name,missed_tackler_team,missed_tackler_numbers,special_teams_safeties_team,special_teams_safeties_numbers,vises_team,vises_numbers,formation_front,formation_mid,formation_back
0,2018090600,37,NaN,NaN,NaN,3.85,D,R,R,NaN,...,Deep,NaN,NaN,PHI,"[23, 27]",NaN,NaN,8.0,0.0,2.0
1,2018090600,366,OK,0.84,2.12,4.46,N,C,C,C,...,Normal,PHI,[57],NaN,NaN,ATL,"[83, 27, 34, 21]",NaN,NaN,NaN
2,2018090600,658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,PHI,[58],NaN,NaN,NaN,NaN,NaN
3,2018090600,677,NaN,NaN,NaN,4.06,D,R,R,C,...,Deep,ATL,[83],ATL,"[17, 22]",NaN,NaN,8.0,0.0,2.0
4,2018090600,872,OK,0.84,2.00,4.35,N,C,L,NaN,...,Normal,NaN,NaN,ATL,[37],ATL,"[83, 34, 21]",NaN,NaN,NaN


**Games**

In [12]:
# before
games.head()

,gameId,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr
0,2018090600,2018,1,09/06/2018,20:20:00,PHI,ATL
1,2018090900,2018,1,09/09/2018,13:00:00,BAL,BUF
2,2018090901,2018,1,09/09/2018,13:00:00,CLE,PIT
3,2018090902,2018,1,09/09/2018,13:00:00,IND,CIN
4,2018090903,2018,1,09/09/2018,13:00:00,MIA,TEN


In [ ]:
# abbreviations: 'homeTeamAbbr' , 'visitorTeamAbbr'
# team number splitting:
# number splitting: 'gameDate', 'gameTimeEastern'

In [ ]:
# abbreviations
# 'homeTeamAbbr', 'visitorTeamAbbr'

In [13]:
# number splitting
# 'gameDate'
csv = games

column = "gameDate"
symbol = "/"
n = 2
column_names = ['game_date_day','game_date_month','game_date_year']
# calling function
game_date_split = number_splitting(csv,column,symbol,n)
# calling function
pd_to_num(csv,game_date_split,column_names)

# 'gameTimeEastern'
column = "gameTimeEastern"
symbol = ":"
n = 2
column_names = ['game_time_hour','game_time_minute','game_time_second']
# calling function
game_time_split = number_splitting(csv,column,symbol,n)
# calling function
pd_to_num(csv,game_time_split,column_names)

In [14]:
# after
games.head()

,gameId,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,game_date_day,game_date_month,game_date_year,game_time_hour,game_time_minute,game_time_second
0,2018090600,2018,1,09/06/2018,20:20:00,PHI,ATL,9,6,2018,20,20,0
1,2018090900,2018,1,09/09/2018,13:00:00,BAL,BUF,9,9,2018,13,0,0
2,2018090901,2018,1,09/09/2018,13:00:00,CLE,PIT,9,9,2018,13,0,0
3,2018090902,2018,1,09/09/2018,13:00:00,IND,CIN,9,9,2018,13,0,0
4,2018090903,2018,1,09/09/2018,13:00:00,MIA,TEN,9,9,2018,13,0,0


**Players**

In [15]:
# before
players.head()

,nflId,height,weight,birthDate,collegeName,Position,displayName
0,42901,6-1,208,1992-07-25,James Madison,SS,Dean Marlowe
1,43501,6-0,220,1994-08-11,Central Michigan,FS,Kavon Frazier
2,43534,6-0,205,1993-06-02,Illinois,SS,Clayton Fejedelem
3,43535,6-1,235,1992-12-22,Temple,MLB,Tyler Matakevich
4,44174,6-1,236,1993-02-22,Texas Tech,ILB,Sam Eguavoen


In [ ]:
# abbreviations:
# team number splitting:
# number splitting: birth
# unique one is height

In [16]:
# 'birthDate'
# splitting the numbers
players['birth_year'] = pd.DatetimeIndex(players['birthDate']).year

In [17]:
# 'height'
height_split = players["height"].str.split("-", n = 1, expand = True)
# new column names
height_split.columns = ['Feet', 'Inch']
# adding new columns
players["height_feet"] = pd.to_numeric(height_split["Feet"])
players["height_inch"] = pd.to_numeric(height_split["Inch"])
# still have 73 feet
# getting rid of NaN values
players["height_inch"] = players["height_inch"].fillna(0)
# replacing heights that did not have '-' at start
for i in players["height_feet"]:
    if i > 7:
        j = 0.0833333 * i
        players["height_feet"].replace({i : j}, inplace=True)        
# inches to numerical value
players["height_inch"] = 0.0833333 * players["height_inch"]
# adding feet and inches to get one value
height_numerical = players["height_feet"] + players["height_inch"]
# adding new numerical value column
players["height_numerical"] = pd.to_numeric(height_numerical)

In [18]:
# after
players.head()

,nflId,height,weight,birthDate,collegeName,Position,displayName,birth_year,height_feet,height_inch,height_numerical
0,42901,6-1,208,1992-07-25,James Madison,SS,Dean Marlowe,1992.0,6.0,0.083333,6.083333
1,43501,6-0,220,1994-08-11,Central Michigan,FS,Kavon Frazier,1994.0,6.0,0.000000,6.000000
2,43534,6-0,205,1993-06-02,Illinois,SS,Clayton Fejedelem,1993.0,6.0,0.000000,6.000000
3,43535,6-1,235,1992-12-22,Temple,MLB,Tyler Matakevich,1992.0,6.0,0.083333,6.083333
4,44174,6-1,236,1993-02-22,Texas Tech,ILB,Sam Eguavoen,1993.0,6.0,0.083333,6.083333


**Plays**

In [19]:
# before
plays.head()

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,specialTeamsPlayType,specialTeamsResult,kickerId,...,penaltyCodes,penaltyJerseyNumbers,penaltyYards,preSnapHomeScore,preSnapVisitorScore,passResult,kickLength,kickReturnYardage,playResult,absoluteYardlineNumber
0,2018090600,37,J.Elliott kicks 65 yards from PHI 35 to end zo...,1,0,0,PHI,Kickoff,Touchback,44966.0,...,NaN,NaN,NaN,0,0,NaN,66.0,NaN,40,45
1,2018090600,366,"(9:20) C.Johnston punts 56 yards to ATL 36, Ce...",1,4,4,PHI,Punt,Return,45603.0,...,UNSd,PHI 18,-15.0,0,0,NaN,56.0,5.0,36,18
2,2018090600,658,"(5:03) M.Bryant 21 yard field goal is GOOD, Ce...",1,4,3,ATL,Field Goal,Kick Attempt Good,27091.0,...,NaN,NaN,NaN,0,0,NaN,21.0,NaN,0,13
3,2018090600,677,M.Bosher kicks 64 yards from ATL 35 to PHI 1. ...,1,0,0,ATL,Kickoff,Return,37267.0,...,NaN,NaN,NaN,0,3,NaN,64.0,30.0,34,75
4,2018090600,872,"(:33) C.Johnston punts 65 yards to end zone, C...",1,4,18,PHI,Punt,Touchback,45603.0,...,NaN,NaN,NaN,0,3,NaN,65.0,NaN,45,45


In [ ]:
# abbreviations: 'possessionTeam'
# team number splitting: 'penaltyJerseyNumbers'
# number splitting: 'gameClock'

In [ ]:
# abbreviations
# 'possessionTeam'

In [ ]:
# team number splitting
# 'penaltyJerseyNumbers'
csv = plays

column = "penaltyJerseyNumbers"
symbol = ";"
# calling function
penaltyJerseyNumbers_team, penaltyJerseyNumbers_numbers = team_number_splitting(csv,column,symbol)
# adding
scouting["penalty_jersey_team"] = penaltyJerseyNumbers_team
scouting["penalty_jersey_numbers"] = penaltyJerseyNumbers_numbers

# broken, think it is to do with values being just team name 'BAL'

In [20]:
# number splitting
# 'gameClock'
csv = plays

column = "gameClock"
symbol = ":"
n = 2
column_names = ['game_clock_minute','game_clock_second','game_clock_milli']
# calling function
game_time_split = number_splitting(csv,column,symbol,n)
# calling function
pd_to_num(csv,game_time_split,column_names)

In [27]:
# adding score difference
plays["pre_snap_difference"] = plays['preSnapHomeScore'] - plays['preSnapVisitorScore']

In [28]:
# after
plays.head()

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,specialTeamsPlayType,specialTeamsResult,kickerId,...,preSnapVisitorScore,passResult,kickLength,kickReturnYardage,playResult,absoluteYardlineNumber,game_clock_minute,game_clock_second,game_clock_milli,pre_snap_difference
0,2018090600,37,J.Elliott kicks 65 yards from PHI 35 to end zo...,1,0,0,PHI,Kickoff,Touchback,44966.0,...,0,NaN,66.0,NaN,40,45,15,0,0,0
1,2018090600,366,"(9:20) C.Johnston punts 56 yards to ATL 36, Ce...",1,4,4,PHI,Punt,Return,45603.0,...,0,NaN,56.0,5.0,36,18,9,20,0,0
2,2018090600,658,"(5:03) M.Bryant 21 yard field goal is GOOD, Ce...",1,4,3,ATL,Field Goal,Kick Attempt Good,27091.0,...,0,NaN,21.0,NaN,0,13,5,3,0,0
3,2018090600,677,M.Bosher kicks 64 yards from ATL 35 to PHI 1. ...,1,0,0,ATL,Kickoff,Return,37267.0,...,3,NaN,64.0,30.0,34,75,5,1,0,-3
4,2018090600,872,"(:33) C.Johnston punts 65 yards to end zone, C...",1,4,18,PHI,Punt,Touchback,45603.0,...,3,NaN,65.0,NaN,45,45,0,33,0,-3


**Tracking**

In [ ]:
# before
tracking2018.head()

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,team,frameId,gameId,playId,playDirection
0,2018-12-30T21:25:32.200,41.32,29.45,4.36,1.33,0.43,130.42,128.44,None,39470.0,Justin Tucker,9.0,K,home,1,2018123000,36,right
1,2018-12-30T21:25:32.300,41.68,29.17,4.59,1.24,0.45,128.59,127.81,None,39470.0,Justin Tucker,9.0,K,home,2,2018123000,36,right
2,2018-12-30T21:25:32.400,42.05,28.88,4.74,0.99,0.47,124.47,128.15,None,39470.0,Justin Tucker,9.0,K,home,3,2018123000,36,right
3,2018-12-30T21:25:32.500,42.43,28.59,4.87,0.71,0.48,126.02,127.35,None,39470.0,Justin Tucker,9.0,K,home,4,2018123000,36,right
4,2018-12-30T21:25:32.600,42.84,28.31,4.96,0.79,0.50,131.71,124.75,None,39470.0,Justin Tucker,9.0,K,home,5,2018123000,36,right


In [ ]:
# it is clean

**Overall, clean now except 'penaltyJerseyNumbers' and some abbrvs**

**Final scores**

In [69]:
final_plays = plays.groupby('gameId').tail(1)

final_scores = final_plays[['preSnapHomeScore','preSnapVisitorScore']]

final_scores.head()

,preSnapHomeScore,preSnapVisitorScore
23,18,12
57,47,3
91,21,21
118,23,34
145,27,20


**Merging**

In [70]:
# sizes
data = [scouting,games,players,plays,tracking2018,tracking2019,tracking2020]
# loop
for name in data:
  print(name.shape)

  # scouting and plays can be merged

(19979, 30)
(764, 13)
(2732, 11)
(19979, 29)
(12777351, 18)
(12170933, 18)
(11821701, 18)


In [71]:
merge = pd.merge(scouting, plays, on = ["gameId","playId"], how = "inner")

merge.head()

,gameId,playId,snapDetail,snapTime,operationTime,hangTime,kickType,kickDirectionIntended,kickDirectionActual,returnDirectionIntended,...,preSnapVisitorScore,passResult,kickLength,kickReturnYardage,playResult,absoluteYardlineNumber,game_clock_minute,game_clock_second,game_clock_milli,pre_snap_difference
0,2018090600,37,NaN,NaN,NaN,3.85,D,R,R,NaN,...,0,NaN,66.0,NaN,40,45,15,0,0,0
1,2018090600,366,OK,0.84,2.12,4.46,N,C,C,C,...,0,NaN,56.0,5.0,36,18,9,20,0,0
2,2018090600,658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,NaN,21.0,NaN,0,13,5,3,0,0
3,2018090600,677,NaN,NaN,NaN,4.06,D,R,R,C,...,3,NaN,64.0,30.0,34,75,5,1,0,-3
4,2018090600,872,OK,0.84,2.00,4.35,N,C,L,NaN,...,3,NaN,65.0,NaN,45,45,0,33,0,-3


In [76]:
# unique values of tracking?
t2018 = tracking2018.groupby('gameId')
t2019 = tracking2019.groupby('gameId')
t2020 = tracking2020.groupby('gameId')

print(len(t2018))
print(len(t2019))
print(len(t2020))
print(len(t2018)+len(t2019)+len(t2020))

# find unique gameIds of all, then can lmerge with games

253
255
256
764


**Field goals**


In [85]:
plays.head(0)

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,specialTeamsPlayType,specialTeamsResult,kickerId,...,preSnapVisitorScore,passResult,kickLength,kickReturnYardage,playResult,absoluteYardlineNumber,game_clock_minute,game_clock_second,game_clock_milli,pre_snap_difference


In [138]:
kick_type = "Field Goal"

field_goal_data = plays.loc[(plays["specialTeamsPlayType"] == kick_type), ["playDescription","gameId","playId","quarter","down","yardsToGo","kickerId","kickLength","playResult",
                                                                       "absoluteYardlineNumber","game_clock_minute","game_clock_second","pre_snap_difference"]]

field_goal_data.head(10)

,playDescription,gameId,playId,quarter,down,yardsToGo,kickerId,kickLength,playResult,absoluteYardlineNumber,game_clock_minute,game_clock_second,pre_snap_difference
2,"(5:03) M.Bryant 21 yard field goal is GOOD, Ce...",2018090600,658,1,4,3,27091.0,21.0,0,13,5,3,0
6,"(6:12) J.Elliott 26 yard field goal is GOOD, C...",2018090600,1368,2,4,8,44966.0,26.0,0,18,6,12,-3
8,"(2:13) M.Bryant 52 yard field goal is GOOD, Ce...",2018090600,1587,2,4,17,27091.0,52.0,0,76,2,13,0
33,(13:18) (Field Goal formation) J.Tucker 41 yar...,2018090900,1226,2,4,13,39470.0,41.0,0,33,13,18,14
36,(8:48) (Field Goal formation) S.Hauschka 52 ya...,2018090900,1613,2,4,9,33469.0,52.0,-8,76,8,48,17
38,(4:20) (Field Goal formation) J.Tucker 39 yard...,2018090900,1973,2,4,2,39470.0,39.0,0,31,4,20,17
50,(2:42) (Field Goal formation) S.Hauschka 35 ya...,2018090900,3461,3,4,17,33469.0,35.0,0,93,2,42,40
90,(1:47) C.Boswell 42 yard field goal is No Good...,2018090901,5191,5,4,3,41953.0,42.0,-8,34,1,47,0
91,(:13) Z.Gonzalez 43 yard field goal is BLOCKED...,2018090901,5480,5,3,11,45037.0,NaN,-8,85,0,13,0
93,"(8:25) R.Bullock 42 yard field goal is GOOD, C...",2018090902,394,1,4,12,38691.0,42.0,0,86,8,25,0


**Scatter plot**

In [136]:
play_ids = field_goal_data["playId"]

**Plot of colour field**

In [135]:
import plotly.graph_objects as go
import plotly.express as px

# figure
fig = go.Figure()

csv = tracking2018
game = 2018090900
play = 1613
# play
play_data = csv.loc[(csv['playId'] == play) & (csv['gameId'] == game)]

# plotting
fig = px.scatter(play_data, 
           x="x", 
           y="y", 
           animation_frame="time", 
           animation_group="nflId",
           color="team",
           hover_name="displayName", 
           width=600, 
           height=400           
          )

# updating axes
fig.update_layout(xaxis_range=[0,120],
                  yaxis_range=[0,53.3],
                  )
# no grid
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

# green rect
fig.add_shape(type="rect",
    x0=10, y0=0, x1=110, y1=53.3,
    line=dict(color="White",
              width=1),
    layer="below",
    fillcolor="LightGreen",
    )
fig.add_shape(type="rect",
    x0=0, y0=0, x1=10, y1=53.3,
    line=dict(color="White",
              width=1),
    layer="below",
    fillcolor="Blue",
    opacity=0.5,
    )
fig.add_shape(type="rect",
    x0=110, y0=0, x1=120, y1=53.3,
    line=dict(color="White",
              width=1),
    layer="below",
    fillcolor="Red",
    opacity=0.5,
    )
# vertical lines
fig.add_shape(type="line",
    x0=10, y0=0, x1=10, y1=53.3,
    layer="below",
    line=dict(color="White",width=2),
    )
fig.add_shape(type="line",
    x0=20, y0=0, x1=20, y1=53.3,
    layer="below",
    line=dict(color="White",width=2),
    )
fig.add_shape(type="line",
    x0=30, y0=0, x1=30, y1=53.3,
    layer="below",
    line=dict(color="White",width=2),
    )
fig.add_shape(type="line",
    x0=40, y0=0, x1=40, y1=53.3,
    layer="below",
    line=dict(color="White",width=2),
    )
fig.add_shape(type="line",
    x0=50, y0=0, x1=50, y1=53.3,
    layer="below",
    line=dict(color="White",width=2),
    )
fig.add_shape(type="line",
    x0=60, y0=0, x1=60, y1=53.3,
    layer="below",
    line=dict(color="White",width=2),
    )
fig.add_shape(type="line",
    x0=70, y0=0, x1=70, y1=53.3,
    layer="below",
    line=dict(color="White",width=2),
    )
fig.add_shape(type="line",
    x0=80, y0=0, x1=80, y1=53.3,
    layer="below",
    line=dict(color="White",width=2),
    )
fig.add_shape(type="line",
    x0=110, y0=0, x1=110, y1=53.3,
    layer="below",
    line=dict(color="White",width=2),
    )
fig.add_shape(type="line",
    x0=90, y0=0, x1=90, y1=53.3,
    layer="below",
    line=dict(color="White",width=2),
    )
fig.add_shape(type="line",
    x0=100, y0=0, x1=100, y1=53.3,
    layer="below",
    line=dict(color="White",width=2),
    )
# horizontal lines
fig.add_shape(type="line",
    x0=10, y0=15, x1=110, y1=15,
    layer="below",
    line=dict(color="White",width=6,
              dash="dot"),
    )
fig.add_shape(type="line",
    x0=10, y0=53.3-15, x1=110, y1=53.3-15,
    layer="below",
    line=dict(color="White",width=6,
              dash="dot"),
    )
# goals
fig.add_shape(type="line",
    x0=1, y0=23.55, x1=1, y1=29.75,
    layer="below",
    line=dict(color="Yellow",width=6),
    )
fig.add_shape(type="line",
    x0=119, y0=23.55, x1=119, y1=29.75,
    layer="below",
    line=dict(color="Yellow",width=6),
    )
# showing
fig.show()